# Project B

## Data preprocessing

Get main shape of the impact and define the criteria to get this part.
Make sure the num. and exp. data are aligned and not in different timeframe.

In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt

In [2]:
def plot4Together(data, title):
    fig = plt.figure()
    fig.suptitle(title)

    axs = fig.subplots(2, 2)

    axs[0, 0].plot(data[:,0], data[:,1], 'tab:blue', label='1')
    axs[0, 0].ticklabel_format(axis='x', style='sci', scilimits=(-4,-4))
    axs[0, 0].yaxis.major.formatter._useMathText = True
    axs[0, 0].legend(loc='best')
    axs[0, 1].plot(data[:,0], data[:,2], 'tab:orange', label='2')
    axs[0, 1].ticklabel_format(axis='x', style='sci', scilimits=(-4,-4))
    axs[0, 1].yaxis.major.formatter._useMathText = True
    axs[0, 1].legend(loc='best')
    axs[1, 0].plot(data[:,0], data[:,3], 'tab:green', label='3')
    axs[1, 0].ticklabel_format(axis='x', style='sci', scilimits=(-4,-4))
    axs[1, 0].yaxis.major.formatter._useMathText = True
    axs[1, 0].legend(loc='best')
    axs[1, 1].plot(data[:,0], data[:,4], 'tab:purple', label='4')
    axs[1, 1].ticklabel_format(axis='x', style='sci', scilimits=(-4,-4))
    axs[1, 1].yaxis.major.formatter._useMathText = True
    axs[1, 1].legend(loc='best')

    fig.subplots_adjust(hspace=0.4)
    plt.show()
    
    pass

### Read data

In [3]:
def readAllNumerical(path):
    subjects = os.listdir(path)
    l = len(subjects)

    label = np.zeros((l, 2), dtype=int)
    dataset = []
    for i in range(l):
        file = scipy.io.loadmat(path + subjects[i])
        
        mat = file['num_data']
        dataset.append(mat)

        label[i, 0] = subjects[i][5:8]
        label[i, 1] = subjects[i][9:12]

    return dataset, label

In [4]:
def readAllExperiment(path):
    dirs = os.listdir(path)
    label = []
    dataset = []
    for j in range(len(dirs)):
        subs = os.listdir(path + '/' + dirs[j])
        for i in range(len(subs)):
            tmp_label = np.zeros(2, dtype=int)

            file = np.transpose(np.loadtxt(path + '/' + dirs[j] + '/' + subs[i], skiprows=8, unpack=True))
            
            # switch 3, 4 columns
            temp = np.copy(file[:, 3])
            file[:, 3] = file[:, 4]
            file[:, 4] = temp
            
            # flip data
            file[:, 1] = -file[:, 1]
            file[:, 2] = -file[:, 2]
            file[:, 3] = -file[:, 3]
            file[:, 4] = -file[:, 4]
            
            dataset.append(file)

            tmp_label[0] = subs[i][10:13]
            tmp_label[1] = subs[i][14:17]
        
            label.append(tmp_label)

    return dataset, label

In [5]:
def readExpTestData(path):
    subjects = os.listdir(path)

    
    # use to replace decimal system DE --> US
    for i in range(len(subjects)):
    #result = subjects[i].replace("'" , "")
    #filename = subjects.append(result)
    with open(path+subjects[i], 'r') as file:
        filedata = file.read()
        filedata = filedata.replace(',','.')

    with open(path+subjects[i],'w') as file:
        file.write(filedata)
    
    dataset = []
    for i in range(len(subjects)):
        
        #file = float(file.replace(',','.'))
        
        file = np.transpose(np.loadtxt(path + '/' + subjects[i], skiprows=8, unpack=True))
        
        # switch 3, 4 columns
        temp = np.copy(file[:, 3])
        file[:, 3] = file[:, 4]
        file[:, 4] = temp
        
        # flip data
        file[:, 1] = -file[:, 1]
        file[:, 2] = -file[:, 2]
        file[:, 3] = -file[:, 3]
        file[:, 4] = -file[:, 4]

        dataset.append(file)

    return dataset

### Cut data

The experiment data is very long hence we cut the data with later defined data points m and n.

In [6]:
def cutWholeData(data, m, n):
    x = data[:, 0]
    y1 = data[:, 1]
    y2 = data[:, 2]
    y3 = data[:, 3]
    y4 = data[:, 4]

    left = np.nonzero(x >  m)
    right = np.nonzero(x <= n)
    interval = np.intersect1d(left, right)

    x_new = x[interval]
    y_new1 = y1[interval]
    y_new2 = y2[interval]
    y_new3 = y3[interval]
    y_new4 = y4[interval]

    data_new = np.vstack([x_new[::10], y_new1[::10], y_new2[::10], y_new3[::10], y_new4[::10]])

    return data_new.T

### Resample experimental data

The numerical data time step, 1e-8, is different from the experiment data. So we resample experiment data to have same time step.

In [7]:
def reSample(data):
    x = data[:, 0]
    y1 = data[:, 1]
    y2 = data[:, 2]
    y3 = data[:, 3]
    y4 = data[:, 4]

    timeline = np.arange(x[0], x[-1], 1e-8)
    f1 = interpolate.interp1d(x, y1, kind='cubic', axis=- 1)
    y_new1 = f1(timeline)
    f2 = interpolate.interp1d(x, y2, kind='cubic', axis=- 1)
    y_new2 = f2(timeline)
    f3 = interpolate.interp1d(x, y3, kind='cubic', axis=- 1)
    y_new3 = f3(timeline)
    f4 = interpolate.interp1d(x, y4, kind='cubic', axis=- 1)
    y_new4 = f4(timeline)

    data_new = np.vstack([timeline, y_new1, y_new2, y_new3, y_new4])

    return data_new.T

### Align numerical and experimental data

We define limit value which sets a starting point where the amplitude is greater than the limit and take certain data points onwards.

In [8]:
def findEnd(y, limit):
    ind = np.nonzero(abs(y) < limit)
    ind_diff = np.diff(ind[0])
    tmp = np.nonzero(ind_diff > 1)
    end = tmp[0][0]

    return end


def alignNumData(data, limit, length, num):
    y1 = data[:, 1]
    y2 = data[:, 2]
    y3 = data[:, 3]
    y4 = data[:, 4]
    
    # find the starting point of the impact shape according to the limit intervall
    end1 = findEnd(y1, limit)
    end2 = findEnd(y2, limit)
    end3 = findEnd(y3, limit)
    end4 = findEnd(y4, limit)

    end = np.min(np.array([end1, end2, end3, end4]))     

    # select the impact area
    y_new1 = y1[end:end + length]
    y_new2 = y2[end:end + length]
    y_new3 = y3[end:end + length]
    y_new4 = y4[end:end + length]

    data_new = np.vstack([y_new1[::num], y_new2[::num], y_new3[::num], y_new4[::num]])

    return data_new.T


def alignExpData(data, m, n, limit, length, num):
    data0 = cutWholeData(data, m, n)
    tmp_data = reSample(data0)

    y1 = tmp_data[:, 1]
    y2 = tmp_data[:, 2]
    y3 = tmp_data[:, 3]
    y4 = tmp_data[:, 4]

    # find the starting point of the impact shape according to the limit intervall
    end1 = findEnd(y1, limit)
    end2 = findEnd(y2, limit)
    end3 = findEnd(y3, limit)
    end4 = findEnd(y4, limit)

    end = np.min(np.array([end1, end2, end3, end4]))

    # select the impact area
    y_new1 = y1[end:end + length]
    y_new2 = y2[end:end + length]
    y_new3 = y3[end:end + length]
    y_new4 = y4[end:end + length]

    data_new = np.vstack([y_new1[::num], y_new2[::num], y_new3[::num], y_new4[::num]])

    return data_new.T

### Augment data to get samples from different quadrants

We rearrange sensor location and make new labels.

In [9]:
def dataAugmentation(dataset, labels):
    ag_dataset = []
    ag_labels = []

    for i in range(len(dataset)):
        # split data for each sensor
        temp = dataset[i]   
        s1 = temp[:,0]
        s2 = temp[:,1]
        s3 = temp[:,2]
        s4 = temp[:,3]

        # rearrange for quadrant 2
        q2 = [s1,s4,s3,s2]
        # compute new labels
        q2_x = 500 - labels[i][0]
        q2_y = labels[i][1]
        l_q2 = [q2_x,q2_y]
        # reshape and store
        q2 = np.array(q2).T
        l_q2 = np.array(l_q2).T       
        ag_dataset.append(q2)
        ag_labels.append(l_q2)

        # rearrange for quadrant 3
        q3 = [s3,s4,s1,s2 ]
        # compute new labels
        q3_x = 500 - labels[i][0]
        q3_y = 500 - labels[i][1]
        l_q3 = [q3_x,q3_y]
        # reshape and store
        q3 = np.array(q3).T
        l_q3 = np.array(l_q3).T       
        ag_dataset.append(q3)
        ag_labels.append(l_q3)

        # rearrange for quadrant 4
        q4 = [s3,s2,s1,s4]
        # compute new labels
        q4_x = labels[i][0]
        q4_y = 500 - labels[i][1]
        l_q4 = [q4_x,q4_y]
        # reshape and store
        q4 = np.array(q4).T
        l_q4 = np.array(l_q4).T       
        ag_dataset.append(q4)
        ag_labels.append(l_q4)
    
    return ag_dataset, ag_labels

### Apply and generate pickle file 

In [10]:
# set limit criterion to assign starting point to sample data of same length
limit = 0.2
# *************************************** NUMERICAL DATA ***************************************
path1 = '../Data/EPOT_Data/'
dataset_num, label_num = readAllNumerical(path1)

# extract data points
outputfile_num = []
for i in range(len(dataset_num)):
    tmp0 = alignNumData(dataset_num[i], limit, 13000, 130)
    outputfile_num.append(tmp0)
    
outputfile_num = np.array(outputfile_num)
print(outputfile_num.shape)
print(label_num.shape)

# save to pickle
#f1 = open('num_input_[244,100,4]_' + str(limit) + '.pickle', 'wb+')
f1 = open('num_input_'+str(outputfile_num.shape)+'_'+ str(limit) + '.pickle', 'wb+')
pickle.dump(outputfile_num, f1)
f1.close()

#f2 = open('num_target_[244,2]_' + str(limit) + '.pickle', 'wb+')
f2 = open('num_target_'+str(label_num.shape)+'_' + str(limit) + '.pickle', 'wb+')
pickle.dump(label_num, f2)
f2.close()

# *************************************** AUGMENTED NUMERICAL DATA ***************************************
path1 = '../Data/EPOT_Data/'
dataset_num, label_num = readAllNumerical(path1)

# extract data points
outputfile_num = []
for i in range(len(dataset_num)):
    tmp0 = alignNumData(dataset_num[i], limit, 13000, 130)
    outputfile_num.append(tmp0)

# data augmentation 
ag_outputfile_num, ag_label_num = dataAugmentation(outputfile_num, label_num)

# merge Data
ag_label_num = np.array(ag_label_num)
label_augnum = np.concatenate((label_num, ag_label_num))
outputfile_num = np.array(outputfile_num)
ag_outputfile_num = np.array(ag_outputfile_num)
outputfile_augnum = np.concatenate((outputfile_num,ag_outputfile_num))

print(outputfile_augnum.shape)
print(label_augnum.shape)

# save to pickle
#f3 = open('num_input_[988,100,4]_' + str(limit) + '.pickle', 'wb+')
f3 = open('num_input_'+str(outputfile_augnum.shape)+'_'+ str(limit) + '.pickle', 'wb+')
pickle.dump(outputfile_augnum, f3)
f3.close()
#f4 = open('num_target_[988,2]_' + str(limit) + '.pickle', 'wb+')
f4 = open('num_target_'+str(label_augnum.shape)+'_' + str(limit) + '.pickle', 'wb+')
pickle.dump(label_augnum, f4)
f4.close()

# *************************************** EXPERIMENTAL DATA ***************************************
path2 = '../Data/Experimental_validation'

dataset_exp, label_exp = readAllExperiment(path2) 

# extract data points
outputfile_exp = []
for i in range(len(dataset_exp)):
    # tmp1 = cutData(dataset_exp[i])
    # tmp2 = reSample(tmp1)
    tmp3 = alignExpData(dataset_exp[i], 4995e-5, 5020e-5, limit, 13000, 130)
    outputfile_exp.append(tmp3)

outputfile_exp = np.array(outputfile_exp)
label_exp = np.array(label_exp)

print(outputfile_exp.shape)
print(label_exp.shape)

# save to pickle
#f5 = open('exp_input_[24,100,4]_' + str(limit) + '.pickle', 'wb+')
f5 = open('exp_input_'+str(outputfile_exp.shape)+'_' + str(limit) + '.pickle', 'wb+')
pickle.dump(outputfile_exp, f5)
f5.close()

#f6 = open('exp_target_[24,2]_' + str(limit) + '.pickle', 'wb+')
f6 = open('exp_target_'+str(label_exp.shape)+'_' + str(limit) + '.pickle', 'wb+')
pickle.dump(label_exp, f6)
f6.close()

# ******************************************* TEST DATA *******************************************
path3 = '../Data/Experimental_data/Group6'
dataset_test = readExpTestData(path3)

outputfile_test = []
for i in range(len(dataset_test)):
    tmp3 = alignExpData(dataset_test[i], 4995e-5, 5020e-5, limit, 13500, 135)
    outputfile_test.append(tmp3)

outputfile_test = np.array(outputfile_test)
print(outputfile_test.shape)

# save to pickle
#f5 = open("test_input_[18,100,4]_d.pickle", 'wb+')
f7 = open('test_input_'+str(outputfile_test.shape)+'_'+ str(limit) + '.pickle', 'wb+')
pickle.dump(outputfile_test, f7)
f7.close()

(244, 100, 4)
(244, 2)
(976, 100, 4)
(976, 2)
(24, 100, 4)
(24, 2)


ValueError: could not convert string to float: '0,00165757'

# Neural Network

In [11]:
from tensorflow import keras
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [12]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

### FIRST RUN: WITHOUT DATA AUGMENTATION (also train, validate and test only on numerical data)

#### Load the data

In [13]:
f = open("num_input_(244, 100, 4)_0.2.pickle",'rb')
x_num = pickle.load(f)
f.close()

f1 = open("num_target_(244, 2)_0.2.pickle",'rb')
y_num = pickle.load(f1)
f1.close()

f2 = open("exp_input_(24, 100, 4)_0.2.pickle", 'rb')
x_exp = pickle.load(f2)
f2.close()

f3 = open("exp_target_(24, 2)_0.2.pickle", 'rb')
y_exp = pickle.load(f3)
f3.close()

Firstly, numerical data will be split into training, validation and test set. Training data is used for training. Validation dataset is used for parameters-tuning for each model. Test set is used to test the performance on numerical data set. 

In [14]:
x_train = x_num[:int(0.8*x_num.shape[0]),:,:]
x_validation = x_num[int(0.8*x_num.shape[0]):int(0.9*x_num.shape[0]),:,:]
x_test = x_num[int(0.9*x_num.shape[0]):,:,:]

y_train = y_num[:int(0.8*y_num.shape[0]),:]
y_validation = y_num[int(0.8*y_num.shape[0]):int(0.9*y_num.shape[0]),:]
y_test = y_num[int(0.9*y_num.shape[0]),:]

print(x_train.shape)
print(x_validation.shape)
print(x_test.shape)

(195, 100, 4)
(24, 100, 4)
(25, 100, 4)


Normalize the data to get mean average and unit covariance.We have found that target data normalization is better for the result, while input data normalization does not make sense. And there is no big difference between MinMaxScaler and StandardScaler.

In [15]:
scaler = MinMaxScaler()
y_trainr = scaler.fit_transform(y_train)

#### The model using Fully Connected Network (FCN)

    FCN model

In [16]:
# This is the best FCN layer we could achieve
# In our experiments, the network with only two layers get the highest accuracy
# More layers do not make sense, even increase the error
# Regularization is very important for better accuracy on validation data. 
# L2 regularization achieves better performance than L1 regularization. 
# Relu activation function is better for hidden layers than tanh activation function
# For the final layer, there is no big difference between 
# No improvement by tring glorot initialization

model_fcn = keras.Sequential([
    keras.layers.Flatten(input_shape=(100, 4)),
    keras.layers.Dense(128,activation='relu',kernel_regularizer = 'l2'),
    keras.layers.Dense(64,activation='relu',kernel_regularizer = 'l2'),
    keras.layers.Dense(2,activation='tanh')
])
model_fcn.compile(optimizer='adam', loss='mse')

In [17]:
model_fcn.fit(x_train, y_trainr, epochs=300, shuffle=True)

Epoch 1/300
7/7 [==============================] - 1s 4ms/step - loss: 3.0690
Epoch 2/300
7/7 [==============================] - 0s 4ms/step - loss: 2.8788
Epoch 3/300
7/7 [==============================] - 0s 3ms/step - loss: 2.7294
Epoch 4/300
7/7 [==============================] - 0s 4ms/step - loss: 2.5930
Epoch 5/300
7/7 [==============================] - 0s 4ms/step - loss: 2.4403
Epoch 6/300
7/7 [==============================] - 0s 3ms/step - loss: 2.3094
Epoch 7/300
7/7 [==============================] - 0s 5ms/step - loss: 2.1746
Epoch 8/300
7/7 [==============================] - 0s 5ms/step - loss: 2.0602
Epoch 9/300
7/7 [==============================] - 0s 3ms/step - loss: 1.9558
Epoch 10/300
7/7 [==============================] - 0s 4ms/step - loss: 1.8644
Epoch 11/300
7/7 [==============================] - 0s 3ms/step - loss: 1.7706
Epoch 12/300
7/7 [==============================] - 0s 4ms/step - loss: 1.6844
Epoch 13/300
7/7 [==============================] - 0s 8ms/st

7/7 [==============================] - 0s 6ms/step - loss: 0.0132
Epoch 207/300
7/7 [==============================] - 0s 5ms/step - loss: 0.0122
Epoch 208/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0115
Epoch 209/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0113
Epoch 210/300
7/7 [==============================] - 0s 2ms/step - loss: 0.0106
Epoch 211/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0099
Epoch 212/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0096
Epoch 213/300
7/7 [==============================] - 0s 4ms/step - loss: 0.0093
Epoch 214/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0091
Epoch 215/300
7/7 [==============================] - 0s 4ms/step - loss: 0.0094
Epoch 216/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0086
Epoch 217/300
7/7 [==============================] - 0s 3ms/step - loss: 0.0079
Epoch 218/300
7/7 [==============================] - 

Get the prediction error for the training data and validation data

In [18]:
predictions = model_fcn.predict(x_train)
predictionsr = scaler.inverse_transform(predictions)
batch_loss = tf.keras.losses.MAE(y_train, predictionsr)
loss_train = np.sum(batch_loss,axis=0) / batch_loss.shape[0]
print("MAE loss on training data",loss_train)

MAE loss on training data 2.819713416466346


In [19]:
predictions1 = model_fcn.predict(x_validation)
predictions1r = scaler.inverse_transform(predictions1)
batch_loss1 = tf.keras.losses.MAE(y_validation, predictions1r)
loss_train1 = np.sum(batch_loss1,axis=0) / x_validation.shape[0]
print("MAE loss on validation data",loss_train1)

MAE loss on validation data 3.3900890350341797


In [20]:
predictions2 = model_fcn.predict(x_test)
predictions2r = scaler.inverse_transform(predictions2)
batch_loss2 = tf.keras.losses.MAE(y_test, predictions2r)
loss_train2 = np.sum(batch_loss2,axis=0) / x_test.shape[0]
print("MAE loss on test data",loss_train2)

MAE loss on test data 24.5577734375


#### The model using Convolutional Neural Network (CNN)

First, expand the dimension of the input data. 

In [21]:
x_trainc = np.expand_dims(x_train,3)
x_validationc = np.expand_dims(x_validation,3)
x_testc = np.expand_dims(x_test,3)

    CNN model

In [22]:
model_cnn = keras.Sequential([
    keras.layers.Conv2D(8,(50,2),activation='relu',input_shape=(100,4,1)),
    keras.layers.MaxPool2D((3,1)),
    keras.layers.Conv2D(8,(10,2),activation='relu'),
    keras.layers.MaxPool2D((2,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(32,activation='relu',kernel_regularizer='l2'),
    keras.layers.Dense(8,activation='relu',kernel_regularizer='l2'),
    keras.layers.Dense(2,activation='tanh')
])
model_cnn.compile(optimizer='adam', loss='mse')

In [23]:
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 51, 3, 8)          808       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 17, 3, 8)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 2, 8)           1288      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 2, 8)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)               

Train the model

In [24]:
model_cnn.fit(x_trainc, y_trainr, epochs=300,shuffle=True)

Epoch 1/300
7/7 [==============================] - 1s 13ms/step - loss: 0.9059
Epoch 2/300
7/7 [==============================] - 0s 11ms/step - loss: 0.7289
Epoch 3/300
7/7 [==============================] - 0s 8ms/step - loss: 0.6818
Epoch 4/300
7/7 [==============================] - 0s 9ms/step - loss: 0.6211
Epoch 5/300
7/7 [==============================] - 0s 10ms/step - loss: 0.5592
Epoch 6/300
7/7 [==============================] - 0s 9ms/step - loss: 0.5174
Epoch 7/300
7/7 [==============================] - 0s 8ms/step - loss: 0.4805
Epoch 8/300
7/7 [==============================] - 0s 10ms/step - loss: 0.4532
Epoch 9/300
7/7 [==============================] - 0s 10ms/step - loss: 0.4279
Epoch 10/300
7/7 [==============================] - 0s 11ms/step - loss: 0.4051
Epoch 11/300
7/7 [==============================] - 0s 11ms/step - loss: 0.3845
Epoch 12/300
7/7 [==============================] - 0s 10ms/step - loss: 0.3645
Epoch 13/300
7/7 [==============================] - 0

7/7 [==============================] - 0s 8ms/step - loss: 0.0142
Epoch 105/300
7/7 [==============================] - 0s 11ms/step - loss: 0.0137
Epoch 106/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0130
Epoch 107/300
7/7 [==============================] - 0s 13ms/step - loss: 0.0129
Epoch 108/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0127
Epoch 109/300
7/7 [==============================] - 0s 11ms/step - loss: 0.0132
Epoch 110/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0125
Epoch 111/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0119
Epoch 112/300
7/7 [==============================] - 0s 7ms/step - loss: 0.0115
Epoch 113/300
7/7 [==============================] - 0s 9ms/step - loss: 0.0113
Epoch 114/300
7/7 [==============================] - 0s 8ms/step - loss: 0.0111
Epoch 115/300
7/7 [==============================] - 0s 8ms/step - loss: 0.0111
Epoch 116/300
7/7 [============================

7/7 [==============================] - 0s 9ms/step - loss: 0.0052
Epoch 207/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0051
Epoch 208/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0051
Epoch 209/300
7/7 [==============================] - 0s 11ms/step - loss: 0.0051
Epoch 210/300
7/7 [==============================] - 0s 9ms/step - loss: 0.0051
Epoch 211/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0052
Epoch 212/300
7/7 [==============================] - 0s 9ms/step - loss: 0.0051
Epoch 213/300
7/7 [==============================] - 0s 9ms/step - loss: 0.0054
Epoch 214/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0051
Epoch 215/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0052
Epoch 216/300
7/7 [==============================] - 0s 10ms/step - loss: 0.0052
Epoch 217/300
7/7 [==============================] - 0s 9ms/step - loss: 0.0054
Epoch 218/300
7/7 [============================

Compute the MAE error on the numerical training and validation data

In [25]:
predictions = model_cnn.predict(x_trainc)
predictionsr = scaler.inverse_transform(predictions)
batch_loss = tf.keras.losses.MAE(y_train, predictionsr)
loss_train = np.sum(batch_loss,axis=0) / x_train.shape[0]
print("MAE loss on training data", loss_train)

MAE loss on training data 2.335072678786058


In [26]:
predictions1 = model_cnn.predict(x_validationc)
predictions1r = scaler.inverse_transform(predictions1)
batch_loss1 = tf.keras.losses.MAE(y_validation, predictions1r)
loss_train1 = np.sum(batch_loss1,axis=0) / x_validation.shape[0]
print("MAE loss on validation data",loss_train1)

MAE loss on validation data 3.1205546061197915


In [27]:
predictions2 = model_cnn.predict(x_testc)
predictions2r = scaler.inverse_transform(predictions2)
batch_loss2 = tf.keras.losses.MAE(y_test, predictions2r)
loss_train2 = np.sum(batch_loss2,axis=0) / x_test.shape[0]
print("MAE loss on test data",loss_train2)

MAE loss on test data 27.24048095703125


#### The model using Recurrent Neural Network (RNN)

The input data for LTSM, the input series cannot be too long compared to FCN and CNN. Therefore, we set the input series to LSTM to be 20. 

In [28]:
x_trainl = x_train[:,::5,:]
x_validationl = x_validation[:,::5,:]
x_testl = x_test[:,::5,:]

    The LSTM model

In [29]:
model_lstm = tf.keras.Sequential()

model_lstm.add(tf.keras.layers.Dense(units=8,activation='relu',kernel_regularizer='l2'))
model_lstm.add(tf.keras.layers.LSTM(units=64,recurrent_initializer='glorot_uniform'))
model_lstm.add(tf.keras.layers.Dropout(rate=0.3))
model_lstm.add(tf.keras.layers.Dense(units=32,activation='relu',kernel_regularizer='l2'))
model_lstm.add(tf.keras.layers.Dense(units=16,activation='relu',kernel_regularizer='l2'))
model_lstm.add(tf.keras.layers.Dense(units=2, activation='tanh'))

model_lstm.compile(loss='msle', optimizer='adam')

Train the model

In [30]:
model_lstm.fit(x_trainl, y_trainr, epochs=300,shuffle=True)

Epoch 1/300
7/7 [==============================] - 6s 27ms/step - loss: 0.7729
Epoch 2/300
7/7 [==============================] - 0s 18ms/step - loss: 0.6900
Epoch 3/300
7/7 [==============================] - 0s 22ms/step - loss: 0.6493
Epoch 4/300
7/7 [==============================] - 0s 19ms/step - loss: 0.6080
Epoch 5/300
7/7 [==============================] - 0s 16ms/step - loss: 0.5685
Epoch 6/300
7/7 [==============================] - 0s 18ms/step - loss: 0.5338
Epoch 7/300
7/7 [==============================] - 0s 18ms/step - loss: 0.5012
Epoch 8/300
7/7 [==============================] - 0s 17ms/step - loss: 0.4715
Epoch 9/300
7/7 [==============================] - 0s 17ms/step - loss: 0.4401
Epoch 10/300
7/7 [==============================] - 0s 17ms/step - loss: 0.4143
Epoch 11/300
7/7 [==============================] - 0s 17ms/step - loss: 0.3877
Epoch 12/300
7/7 [==============================] - 0s 15ms/step - loss: 0.3623
Epoch 13/300
7/7 [==============================]

7/7 [==============================] - 0s 16ms/step - loss: 0.0073
Epoch 205/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0075
Epoch 206/300
7/7 [==============================] - 0s 20ms/step - loss: 0.0070
Epoch 207/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0069
Epoch 208/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0066
Epoch 209/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0070
Epoch 210/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0071
Epoch 211/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0066
Epoch 212/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0067
Epoch 213/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0067
Epoch 214/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0068
Epoch 215/300
7/7 [==============================] - 0s 16ms/step - loss: 0.0067
Epoch 216/300
7/7 [=======================

The loss on training and validation data

In [31]:
predictions = model_lstm.predict(x_trainl)
predictionsr = scaler.inverse_transform(predictions)
batch_loss = tf.keras.losses.MAE(y_train, predictionsr)
loss_train = np.sum(batch_loss,axis=0) / x_trainl.shape[0]
print("MAE loss on training data", loss_train)

MAE loss on training data 4.081645633012821


In [32]:
predictions1 = model_lstm.predict(x_validationl)
predictions1r = scaler.inverse_transform(predictions1)
batch_loss1 = tf.keras.losses.MAE(y_validation, predictions1r)
loss_train1 = np.sum(batch_loss1,axis=0) / x_validationl.shape[0]
print("MAE loss on validation data",loss_train1)

MAE loss on validation data 4.37278938293457


In [33]:
predictions2 = model_lstm.predict(x_testl)
predictions2r = scaler.inverse_transform(predictions2)
batch_loss2 = tf.keras.losses.MAE(y_test, predictions2r)
loss_train2 = np.sum(batch_loss2,axis=0) / x_test.shape[0]
print("MAE loss on test data",loss_train2)

MAE loss on test data 27.01715087890625


    GRU model

We could also try another RNN model using gated recurrent unit (GRU). 

In [34]:
model_gru = tf.keras.Sequential()

model_gru.add(tf.keras.layers.Dense(units=8,activation='relu',kernel_regularizer='l2'))
model_gru.add(tf.keras.layers.GRU(units=64,recurrent_initializer='glorot_uniform'))
model_gru.add(tf.keras.layers.Dropout(rate=0.3))
model_gru.add(tf.keras.layers.Dense(units=32,activation='relu',kernel_regularizer='l2'))
model_gru.add(tf.keras.layers.Dense(units=16,activation='relu',kernel_regularizer='l2'))
model_gru.add(tf.keras.layers.Dense(units=2, activation='tanh'))

model_gru.compile(loss='msle', optimizer='adam')

In [35]:
model_gru.fit(x_trainl, y_trainr, epochs=300,shuffle=True)

Epoch 1/300
7/7 [==============================] - 6s 21ms/step - loss: 0.8273
Epoch 2/300
7/7 [==============================] - 0s 17ms/step - loss: 0.7486
Epoch 3/300
7/7 [==============================] - 0s 18ms/step - loss: 0.6884
Epoch 4/300
7/7 [==============================] - 0s 18ms/step - loss: 0.6347
Epoch 5/300
7/7 [==============================] - 0s 18ms/step - loss: 0.5931
Epoch 6/300
7/7 [==============================] - 0s 17ms/step - loss: 0.5575
Epoch 7/300
7/7 [==============================] - 0s 17ms/step - loss: 0.5271
Epoch 8/300
7/7 [==============================] - 0s 16ms/step - loss: 0.4924
Epoch 9/300
7/7 [==============================] - 0s 17ms/step - loss: 0.4650
Epoch 10/300
7/7 [==============================] - 0s 17ms/step - loss: 0.4375
Epoch 11/300
7/7 [==============================] - 0s 17ms/step - loss: 0.4123
Epoch 12/300
7/7 [==============================] - 0s 18ms/step - loss: 0.3864
Epoch 13/300
7/7 [==============================]

7/7 [==============================] - 0s 18ms/step - loss: 0.0092
Epoch 205/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0088
Epoch 206/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0092
Epoch 207/300
7/7 [==============================] - 0s 16ms/step - loss: 0.0089
Epoch 208/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0088
Epoch 209/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0090
Epoch 210/300
7/7 [==============================] - 0s 18ms/step - loss: 0.0090
Epoch 211/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0092
Epoch 212/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0092
Epoch 213/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0091
Epoch 214/300
7/7 [==============================] - 0s 15ms/step - loss: 0.0087
Epoch 215/300
7/7 [==============================] - 0s 17ms/step - loss: 0.0083
Epoch 216/300
7/7 [=======================

The loss on training and validation data

In [36]:
predictions = model_gru.predict(x_trainl)
predictionsr = scaler.inverse_transform(predictions)
batch_loss = tf.keras.losses.MAE(y_train, predictionsr)
loss_train = np.sum(batch_loss,axis=0) / x_trainl.shape[0]
print("MAE loss on training data", loss_train)

MAE loss on training data 4.625798152043269


In [37]:
predictions1 = model_gru.predict(x_validationl)
predictions1r = scaler.inverse_transform(predictions1)
batch_loss1 = tf.keras.losses.MAE(y_validation, predictions1r)
loss_train1 = np.sum(batch_loss1,axis=0) / x_validationl.shape[0]
print("MAE loss on validation data",loss_train1)

MAE loss on validation data 5.448814392089844


In [38]:
predictions2 = model_gru.predict(x_testl)
predictions2r = scaler.inverse_transform(predictions2)
batch_loss2 = tf.keras.losses.MAE(y_test, predictions2r)
loss_train2 = np.sum(batch_loss2,axis=0) / x_testl.shape[0]
print("MAE loss on test data",loss_train2)

MAE loss on test data 25.828017578125


### SECOND RUN (train on augmented numerical data, test on experimental_validation data)

#### Data augmentation for numerical data

In [39]:
fa = open("num_input_(976, 100, 4)_0.2.pickle",'rb')
x_numa = pickle.load(fa)
fa.close()

f1a = open("num_target_(976, 2)_0.2.pickle",'rb')
y_numa = pickle.load(f1a)
f1a.close()

In [40]:
scalera = MinMaxScaler()
y_traina = scalera.fit_transform(y_numa)

    The prediction of FCN on experimental data

In [41]:
model_fcn.fit(x_numa, y_traina, epochs=300, shuffle=True)

Epoch 1/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0822
Epoch 2/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0317
Epoch 3/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0201
Epoch 4/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0163
Epoch 5/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0160
Epoch 6/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0156
Epoch 7/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0143
Epoch 8/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0141
Epoch 9/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0144
Epoch 10/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0140
Epoch 11/300
31/31 [==============================] - ETA: 0s - loss: 0.013 - 0s 3ms/step - loss: 0.0136
Epoch 12/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0143
Epoch 13/300


31/31 [==============================] - 0s 3ms/step - loss: 0.0088
Epoch 201/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0087
Epoch 202/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0076
Epoch 203/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0073
Epoch 204/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 205/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0075
Epoch 206/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0079
Epoch 207/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0077
Epoch 208/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0076
Epoch 209/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0076
Epoch 210/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0072
Epoch 211/300
31/31 [==============================] - 0s 3ms/step - loss: 0.0068
Epoch 212/300
31/31 [=========

31/31 [==============================] - 0s 2ms/step - loss: 0.0076


In [42]:
predictions3 = model_fcn.predict(x_exp)
predictions3r = scalera.inverse_transform(predictions3)
batch_loss3 = tf.keras.losses.MAE(y_exp, predictions3r)
loss_train3 = np.sum(batch_loss3,axis=0) / x_exp.shape[0]
print("MAE loss on experimental data",loss_train3)

MAE loss on experimental data 10.78652826944987


    Prediction of CNN on experimental data

In [43]:
x_numac = np.expand_dims(x_numa,3)

In [44]:
model_cnn.fit(x_numac, y_traina, epochs=300,shuffle=True)

Epoch 1/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0381
Epoch 2/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0131
Epoch 3/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0092
Epoch 4/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0077
Epoch 5/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0068
Epoch 6/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0063
Epoch 7/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0062
Epoch 8/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0057
Epoch 9/300
31/31 [==============================] - 0s 8ms/step - loss: 0.0054
Epoch 10/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0052
Epoch 11/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0050
Epoch 12/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0047
Epoch 13/300
31/31 [=================

31/31 [==============================] - 0s 7ms/step - loss: 0.0010
Epoch 203/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 204/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0010
Epoch 205/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0010
Epoch 206/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0010
Epoch 207/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 208/300
31/31 [==============================] - 0s 8ms/step - loss: 0.0012
Epoch 209/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 210/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 211/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 212/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 213/300
31/31 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 214/300
31/31 [=========

In [45]:
x_expc = np.expand_dims(x_exp,3)

In [46]:
predictions3 = model_cnn.predict(x_expc)
predictions3r = scalera.inverse_transform(predictions3)
batch_loss3 = tf.keras.losses.MAE(y_exp, predictions3r)
loss_train3 = np.sum(batch_loss3,axis=0) / x_exp.shape[0]
print("MAE loss on experimental data",loss_train3)

MAE loss on experimental data 15.443712870279947


    Predicion of RNN using LSTM on experimental data

In [47]:
x_numal = x_numa[:,::5,:]

In [48]:
model_lstm.fit(x_numal, y_traina, epochs=300, shuffle=True)

Epoch 1/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0322
Epoch 2/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0162
Epoch 3/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0116
Epoch 4/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0102
Epoch 5/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0094
Epoch 6/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0089
Epoch 7/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0087
Epoch 8/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0081
Epoch 9/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0079
Epoch 10/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0078
Epoch 11/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0075
Epoch 12/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0073
Epoch 13/300
31/31 [=====

31/31 [==============================] - 0s 14ms/step - loss: 0.0025
Epoch 199/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0026
Epoch 200/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0024
Epoch 201/300
31/31 [==============================] - 0s 13ms/step - loss: 0.0027
Epoch 202/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0027
Epoch 203/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0026
Epoch 204/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0026
Epoch 205/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0025
Epoch 206/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0024
Epoch 207/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0026
Epoch 208/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0024
Epoch 209/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0029
Epoch 210/300
31/3

In [49]:
x_expl = x_exp[:,::5,:]

In [50]:
predictions3 = model_lstm.predict(x_expl)
predictions3r = scalera.inverse_transform(predictions3)
batch_loss3 = tf.keras.losses.MAE(y_exp, predictions3r)
loss_train3 = np.sum(batch_loss3,axis=0) / x_exp.shape[0]
print("MAE loss on experimental data",loss_train3)

MAE loss on experimental data 7.530952453613281


    Prediction of RNN using GRU on experimental data

In [51]:
model_gru.fit(x_numal, y_traina, epochs=300,shuffle=True)

Epoch 1/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0365
Epoch 2/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0292
Epoch 3/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0288
Epoch 4/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0278
Epoch 5/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0274
Epoch 6/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0270
Epoch 7/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0264
Epoch 8/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0221
Epoch 9/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0174
Epoch 10/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0130
Epoch 11/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0102
Epoch 12/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0094
Epoch 13/300
31/31 [=====

31/31 [==============================] - 0s 15ms/step - loss: 0.0034
Epoch 102/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0037
Epoch 103/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0033
Epoch 104/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0033
Epoch 105/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0033
Epoch 106/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0034
Epoch 107/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0034
Epoch 108/300
31/31 [==============================] - 0s 14ms/step - loss: 0.0034
Epoch 109/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0035
Epoch 110/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0032
Epoch 111/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0033
Epoch 112/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0031
Epoch 113/300
31/3

31/31 [==============================] - 0s 8ms/step - loss: 0.0023
Epoch 299/300
31/31 [==============================] - 0s 8ms/step - loss: 0.0025
Epoch 300/300
31/31 [==============================] - 0s 8ms/step - loss: 0.0025


In [52]:
predictions3 = model_gru.predict(x_expl)
predictions3r = scalera.inverse_transform(predictions3)
batch_loss3 = tf.keras.losses.MAE(y_exp, predictions3r)
loss_train3 = np.sum(batch_loss3,axis=0) / x_exp.shape[0]
print("MAE loss on experimental data",loss_train3)

MAE loss on experimental data 6.024421691894531


### Test on experimental data given to Group5

In [53]:
ff = open("test_input_(18, 100, 4)_0.2.pickle",'rb')
x_final = pickle.load(ff)
ff.close()

In [54]:
x_final2 = x_final[:,::5,:]

    We use GRU model to get our final prediction

In [55]:
final_predict = model_gru.predict(x_final2)
final_result = scalera.inverse_transform(final_predict)
print("This is our prediction to the experimental data given to us:")
print(final_result)

This is our prediction to the experimental data given to us:
[[249.59981 245.85228]
 [224.03537 210.05815]
 [256.09964 225.64336]
 [258.41632 299.2416 ]
 [185.49352 235.54468]
 [260.58017 241.13808]
 [241.02893 308.2439 ]
 [247.59097 255.41391]
 [270.94785 308.19196]
 [233.56549 228.30408]
 [244.77493 250.3013 ]
 [283.37823 279.81262]
 [247.65222 249.45523]
 [234.62479 234.90514]
 [246.80484 246.08168]
 [238.74994 245.78639]
 [246.63173 246.91553]
 [260.8678  215.07516]]


In [56]:
actual = np.matrix([[250, 265],
                   [265, 235],
                   [235, 235],
                   [245, 255],
                   [255, 265],
                   [250, 250],
                   [200, 225],
                   [225, 200],
                   [270, 300],
                   [250, 250],
                   [265, 235],
                   [235, 300],
                   [265, 255],
                   [235, 235],
                   [250, 265],
                   [260, 320],
                   [245, 255],
                   [300, 285]])

In [57]:
print(final_result.shape)
print(actual.shape)

(18, 2)
(18, 2)


In [58]:
#prediction == actual
result = np.absolute(actual - final_result)
print(result)

[[ 0.40019226 19.14772034]
 [40.96463013 24.94184875]
 [21.09963989  9.35664368]
 [13.4163208  44.24160767]
 [69.50648499 29.45532227]
 [10.58016968  8.86192322]
 [41.02893066 83.24389648]
 [22.5909729  55.41390991]
 [ 0.94784546  8.19195557]
 [16.43450928 21.69592285]
 [20.22506714 15.30130005]
 [48.37823486 20.18737793]
 [17.34777832  5.54476929]
 [ 0.37521362  0.09486389]
 [ 3.19515991 18.9183197 ]
 [21.25006104 74.21360779]
 [ 1.63172913  8.08447266]
 [39.13220215 69.92483521]]


In [59]:
from sklearn.metrics import mean_absolute_error as MAE
MAE(actual, final_result)

25.147928873697914